In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn import preprocessing
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.model_selection import GridSearchCV

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader,Dataset
from torch.autograd import Variable

torch.manual_seed(0)

In [2]:
#Load tabular data
Dataset=pd.read_csv(r'data/TabularSummaryBatteryData.csv')

#Scale Cell life target Values
Dataset['cell_life']=np.log10(Dataset['cell_life'])

numBat1=41
numBat2=43
numBat3=40
numBat=124
test_ind = np.hstack((np.arange(0,(numBat1+numBat2),2),83))
test_edited_ind = np.delete(test_ind, 21) # Remove battery 
train_ind = np.arange(1,(numBat1+numBat2-1),2)
secondary_test_ind = np.arange(numBat-numBat3,numBat)

Training_Data=Dataset.iloc[train_ind]
Testing_Data=Dataset.iloc[test_edited_ind]
Secondary_Data=Dataset.iloc[secondary_test_ind]

#choosing model features According to table 1
target_features=['cell_life']

varmod_features=['variance_dQ_100_10']

dismod_features=['minimum_dQ_100_10',
                 'variance_dQ_100_10',
                 'skewness_dQ_100_10',
                 'kurtosis_dQ_100_10',
                 'Discharge_Cap_2',
                 'Diff_MaxCap_DC2']
    
full_features=['minimum_dQ_100_10',
               'variance_dQ_100_10',
               'Slope_Cap_Fade_2_100',
               'Intercept_Cap_Fade_2_100',
               'Diff_MaxCap_DC2',
               'Discharge_Cap_2',
               'Avg_charge_time',
               'Temp_Integ_2_100',
               'Min_IR',
               'Diff_IR_2_100']

In [3]:
#Use only the discharge model Tabular features
features = dismod_features #full_features
X_train=Training_Data[features].to_numpy()
X_test=Testing_Data[features].to_numpy()
X_sec=Secondary_Data[features].to_numpy()

#Scale
scaler = preprocessing.StandardScaler().fit(X_train)
Tabular_Train = scaler.transform(X_train)
Tabular_test = scaler.transform(X_test)
Tabular_sec = scaler.transform(X_sec)

In [4]:
#Load discharge capacity data
train_Qdlin=torch.load(r'data/DischargeCapacity-InterpolatedData_Train.pt')
test_Qdlin=torch.load(r'data/DischargeCapacity-InterpolatedData_Test.pt')
secondary_test_Qdlin=torch.load(r'data/DischargeCapacity-InterpolatedData_Secondary_Test.pt')

train_Qdlin = torch.permute(train_Qdlin, (0, 2, 1))
test_Qdlin = torch.permute(test_Qdlin, (0, 2, 1))
secondary_test_Qdlin = torch.permute(secondary_test_Qdlin, (0, 2, 1))

print(train_Qdlin.std(),train_Qdlin.mean())
print(test_Qdlin.std(),test_Qdlin.mean())
print(secondary_test_Qdlin.std(),secondary_test_Qdlin.mean())

print(train_Qdlin.shape)
print(test_Qdlin.shape)
print(secondary_test_Qdlin.shape)

tensor(1.0000, dtype=torch.float64) tensor(-0.6982, dtype=torch.float64)
tensor(0.8786, dtype=torch.float64) tensor(-0.6188, dtype=torch.float64)
tensor(0.4862, dtype=torch.float64) tensor(-0.3183, dtype=torch.float64)
torch.Size([41, 99, 100])
torch.Size([42, 99, 100])
torch.Size([40, 99, 100])


In [5]:
y_train=Training_Data[target_features].to_numpy()
y_test=Testing_Data[target_features].to_numpy()
y_sec=Secondary_Data[target_features].to_numpy()

y_train = torch.tensor(y_train)
y_test = torch.tensor(y_test)
y_sec = torch.tensor(y_sec)

y_train_mean = torch.mean(y_train)
y_train = y_train - y_train_mean
y_test = y_test - y_train_mean
y_sec = y_sec - y_train_mean
#Be sure to add y_train_mean and 10** All results

In [6]:
from torch.utils.data import Dataset
class CustomDataset(Dataset):
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets
    
    def __getitem__(self, idx):
        return self.features[idx], self.targets[idx]
  
    def __len__(self):
        return len(self.features)

In [7]:
class LSTMRNN(nn.Module):

    def __init__(self, input_size, hidden_size, hidden_size2, num_layers,
                 dropout, num_dense_neurons):
        
        super(LSTMRNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.hidden_size2 = hidden_size2
        self.num_layers = num_layers
        self.dropout = dropout
        self.num_dense_neurons = num_dense_neurons
        self.firstEpochFlag = True
        
        self.dropout_layer = nn.Dropout(self.dropout)
        
        self.lstm = nn.LSTM(input_size = self.input_size,
                            hidden_size = self.hidden_size,
                            num_layers = self.num_layers,
#                             dropout = self.dropout,
                            batch_first = True)
        
        self.lstm2 = nn.LSTM(input_size = self.hidden_size,
                            hidden_size = self.hidden_size2,
                            num_layers = self.num_layers,
#                             dropout = self.dropout,
                            batch_first = True)
        
        self.linear_layer = nn.Sequential(
            #nn.Dropout(self.dropout)
            nn.Linear(self.hidden_size2, self.num_dense_neurons),
            nn.ReLU(),
            nn.Linear(num_dense_neurons,1),
        )
  
    def forward(self, x):
        if(self.firstEpochFlag):print(x.shape)
        h0 = Variable(torch.zeros(self.num_layers, x.shape[0], self.hidden_size,device=x.device))
        c0 = Variable(torch.zeros(self.num_layers, x.shape[0], self.hidden_size,device=x.device))
        lstm_output,(h1, c1) = self.lstm(x,(h0, c0))
        if(self.firstEpochFlag):print(lstm_output.shape)
            
        h00 = Variable(torch.zeros(self.num_layers, x.shape[0], self.hidden_size2,device=x.device))
        c00 = Variable(torch.zeros(self.num_layers, x.shape[0], self.hidden_size2,device=x.device))
        lstm_output2,(h2, c2) = self.lstm2(lstm_output,(h00, c00))
        if(self.firstEpochFlag):print(lstm_output2.shape)
            
        flat = lstm_output2[:, -1, :]#last projected timestep (cycle)
        if(self.firstEpochFlag):print(flat.shape)
        
        output = self.linear_layer(flat)
        if(self.firstEpochFlag):
            (output.shape)
            self.firstEpochFlag=False
        return output

In [8]:
def train(model,train_loader,criterion,optimizer,epochs,device):
    for epoch in range(epochs):
        model.train()
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            inputs = inputs.to(device)
            targets = targets.to(device)
            
            outputs = model(inputs)
            outputs.to(device)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
        model.eval()
        if(epoch%100 == 0):
            print('--',epoch,'-epoch--RMSE-- ',
                  np.sqrt(mean_squared_error(10**(targets.cpu().detach().numpy()+y_train_mean.numpy()), 
                                          10**(outputs.cpu().detach().numpy()+y_train_mean.numpy()))))
    return model

In [9]:
def testALL(model):
    dataloaders = [['---RMSE---Train',train_Qdlin_loader],
                   ['---RMSE---Test',test_Qdlin_loader],
                   ['---RMSE---Test Secondary',secondary_test_Qdlin_loader]]
    for dataloader in dataloaders:
        for inputs, targets in dataloader[1]:
            inputs = inputs.to(device)
            targets = targets.to(device)
            
            outputs = model(inputs)
            outputs.to(device)
            loss = criterion(outputs, targets)
            print(np.sqrt(mean_squared_error(10**(targets.cpu().detach().numpy()+y_train_mean.numpy()), 
                                             10**(outputs.cpu().detach().numpy()+y_train_mean.numpy()))),dataloader[0])

In [10]:
torch.set_default_dtype(torch.float64)

In [11]:
train_Dataset = CustomDataset(train_Qdlin,y_train)
test_Dataset = CustomDataset(test_Qdlin,y_test)
secondary_test_Dataset = CustomDataset(secondary_test_Qdlin,y_sec)

train_Qdlin_loader = DataLoader(train_Dataset, batch_size=41, shuffle=True)
test_Qdlin_loader = DataLoader(test_Dataset, batch_size=42, shuffle=True)
secondary_test_Qdlin_loader = DataLoader(secondary_test_Dataset, batch_size=40, shuffle=True)

# Hyper-parameters

In [12]:
hidden_size = 128 
hidden_size2 = 256
num_layers = 1
dropout = .2
num_dense_neurons= 32

epochs = 1500
lr = 0.0001
weight_decay = 0.001

In [13]:
input_size=train_Qdlin.shape[2] #FEATURES
num_cycles=train_Qdlin.shape[1] #TIMESTEPS(Cycles)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")
print(device)

cuda:0


# Train & Test

In [14]:
criterion=nn.MSELoss()
model=LSTMRNN(input_size, hidden_size, hidden_size2, num_layers,dropout, num_dense_neurons)
optimizer=optim.Adam(model.parameters(),weight_decay=weight_decay, lr=lr)
model = model.to(device)
model = train(model,train_Qdlin_loader,criterion,optimizer,epochs,device)

torch.Size([41, 99, 100])
torch.Size([41, 99, 128])
torch.Size([41, 99, 256])
torch.Size([41, 256])
-- 0 -epoch--RMSE--  325.713504214364
-- 100 -epoch--RMSE--  191.60093282624348
-- 200 -epoch--RMSE--  118.58940215229036
-- 300 -epoch--RMSE--  84.2295624764314
-- 400 -epoch--RMSE--  65.92709414591444
-- 500 -epoch--RMSE--  52.18300708080519
-- 600 -epoch--RMSE--  46.43477470967451
-- 700 -epoch--RMSE--  43.97749221237031
-- 800 -epoch--RMSE--  42.366674639828105
-- 900 -epoch--RMSE--  38.88688856498636
-- 1000 -epoch--RMSE--  37.52256793585603
-- 1100 -epoch--RMSE--  36.52446165852783
-- 1200 -epoch--RMSE--  35.78776480837691
-- 1300 -epoch--RMSE--  35.32158666670042
-- 1400 -epoch--RMSE--  34.85211871891183


In [15]:
testALL(model)

35.13341232095763 ---RMSE---Train
101.92579699870431 ---RMSE---Test
193.827633377642 ---RMSE---Test Secondary


################################################################################

# Repeat using only the first 60 cycles

################################################################################

In [16]:
#Load tabular data
Dataset=pd.read_csv(r'data/TabularSummaryBatteryData-60Cycles.csv')

# Scale target Battery life By log 10
Dataset['cell_life']=np.log10(Dataset['cell_life'])

Training_Data=Dataset.iloc[train_ind]
Testing_Data=Dataset.iloc[test_edited_ind]
Secondary_Data=Dataset.iloc[secondary_test_ind]

#choosing model features According to table 1
target_features=['cell_life']

varmod_features=['variance_dQ_60_10']

dismod_features=['minimum_dQ_60_10',
                 'variance_dQ_60_10',
                 'skewness_dQ_60_10',
                 'kurtosis_dQ_60_10',
                 'Discharge_Cap_2',
                 'Diff_MaxCap_DC2']
    
full_features=['minimum_dQ_60_10',
               'variance_dQ_60_10',
               'Slope_Cap_Fade_2_60',
               'Intercept_Cap_Fade_2_60',
               'Diff_MaxCap_DC2',
               'Discharge_Cap_2',
               'Avg_charge_time',
               'Temp_Integ_2_60',
               'Min_IR',
               'Diff_IR_2_60']

In [17]:
#Use only the discharge model Tabular features
features = dismod_features #full_features
X_train=Training_Data[features].to_numpy()
X_test=Testing_Data[features].to_numpy()
X_sec=Secondary_Data[features].to_numpy()

#Scale
scaler = preprocessing.StandardScaler().fit(X_train)
Tabular_Train = scaler.transform(X_train)
Tabular_test = scaler.transform(X_test)
Tabular_sec = scaler.transform(X_sec)

In [18]:
#Load discharge capacity data
train_Qdlin=torch.load(r'data/DischargeCapacity-InterpolatedData_Train.pt')
test_Qdlin=torch.load(r'data/DischargeCapacity-InterpolatedData_Test.pt')
secondary_test_Qdlin=torch.load(r'data/DischargeCapacity-InterpolatedData_Secondary_Test.pt')

train_Qdlin = train_Qdlin[:,:60,:]
test_Qdlin = test_Qdlin[:,:60,:]
secondary_test_Qdlin = secondary_test_Qdlin[:,:60,:]

train_Qdlin = torch.permute(train_Qdlin, (0, 2, 1))
test_Qdlin = torch.permute(test_Qdlin, (0, 2, 1))
secondary_test_Qdlin = torch.permute(secondary_test_Qdlin, (0, 2, 1))

print(train_Qdlin.std(),train_Qdlin.mean())
print(test_Qdlin.std(),test_Qdlin.mean())
print(secondary_test_Qdlin.std(),secondary_test_Qdlin.mean())

print(train_Qdlin.shape)
print(test_Qdlin.shape)
print(secondary_test_Qdlin.shape)

tensor(1.1621) tensor(-0.8226)
tensor(1.0245) tensor(-0.7291)
tensor(0.5661) tensor(-0.3582)
torch.Size([41, 99, 60])
torch.Size([42, 99, 60])
torch.Size([40, 99, 60])


In [19]:
y_train=Training_Data[target_features].to_numpy()
y_test=Testing_Data[target_features].to_numpy()
y_sec=Secondary_Data[target_features].to_numpy()

y_train = torch.tensor(y_train)
y_test = torch.tensor(y_test)
y_sec = torch.tensor(y_sec)

y_train_mean = torch.mean(y_train)
y_train = y_train - y_train_mean
y_test = y_test - y_train_mean
y_sec = y_sec - y_train_mean
#Be sure to add y_train_mean and 10** All results

In [20]:
torch.set_default_dtype(torch.float64)

In [21]:
train_Dataset = CustomDataset(train_Qdlin,y_train)
test_Dataset = CustomDataset(test_Qdlin,y_test)
secondary_test_Dataset = CustomDataset(secondary_test_Qdlin,y_sec)

train_Qdlin_loader = DataLoader(train_Dataset, batch_size=41, shuffle=True)
test_Qdlin_loader = DataLoader(test_Dataset, batch_size=42, shuffle=True)
secondary_test_Qdlin_loader = DataLoader(secondary_test_Dataset, batch_size=40, shuffle=True)

# Hyper-parameters

In [22]:
hidden_size = 128 
hidden_size2 = 256
num_layers = 1
dropout = .2
num_dense_neurons= 32

epochs = 1500
lr = 0.0001
weight_decay = 0.001

In [23]:
input_size=train_Qdlin.shape[2] #FEATURES
num_cycles=train_Qdlin.shape[1] #TIMESTEPS(Cycles)

#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")
print(device)

cuda:0


# Train & Test

In [24]:
criterion=nn.MSELoss()
model=LSTMRNN(input_size, hidden_size, hidden_size2, num_layers,dropout, num_dense_neurons)
optimizer=optim.Adam(model.parameters(),weight_decay=weight_decay, lr=lr)
model = model.to(device)
model = train(model,train_Qdlin_loader,criterion,optimizer,epochs,device)

torch.Size([41, 99, 60])
torch.Size([41, 99, 128])
torch.Size([41, 99, 256])
torch.Size([41, 256])
-- 0 -epoch--RMSE--  336.980383494875
-- 100 -epoch--RMSE--  196.49968004765717
-- 200 -epoch--RMSE--  131.85181522835035
-- 300 -epoch--RMSE--  94.58795444219324
-- 400 -epoch--RMSE--  76.86408492435504
-- 500 -epoch--RMSE--  67.48869808237947
-- 600 -epoch--RMSE--  59.78603669183872
-- 700 -epoch--RMSE--  55.16332345419964
-- 800 -epoch--RMSE--  51.36938232273512
-- 900 -epoch--RMSE--  73.4541346743964
-- 1000 -epoch--RMSE--  45.78932029781354
-- 1100 -epoch--RMSE--  42.95469577445578
-- 1200 -epoch--RMSE--  42.12922462716005
-- 1300 -epoch--RMSE--  40.427754874988594
-- 1400 -epoch--RMSE--  39.86140423345441


In [25]:
testALL(model)

38.01373387863486 ---RMSE---Train
316.0990761642964 ---RMSE---Test
220.24435392175144 ---RMSE---Test Secondary


################################################################################

# Repeat using only the first 40 cycles

################################################################################

In [26]:
#Load tabular data
Dataset=pd.read_csv(r'data/TabularSummaryBatteryData-40Cycles.csv')

# Scale target Battery life By log 10
Dataset['cell_life']=np.log10(Dataset['cell_life'])

numBat1=41
numBat2=43
numBat3=40
numBat=124
test_ind = np.hstack((np.arange(0,(numBat1+numBat2),2),83))
test_edited_ind = np.delete(test_ind, 21) # Remove battery 
train_ind = np.arange(1,(numBat1+numBat2-1),2)
secondary_test_ind = np.arange(numBat-numBat3,numBat)

Training_Data=Dataset.iloc[train_ind]
Testing_Data=Dataset.iloc[test_edited_ind]
Secondary_Data=Dataset.iloc[secondary_test_ind]

#choosing model features According to table 1
target_features=['cell_life']

varmod_features=['variance_dQ_40_10']

dismod_features=['minimum_dQ_40_10',
                 'variance_dQ_40_10',
                 'skewness_dQ_40_10',
                 'kurtosis_dQ_40_10',
                 'Discharge_Cap_2',
                 'Diff_MaxCap_DC2']
    
full_features=['minimum_dQ_40_10',
               'variance_dQ_40_10',
               'Slope_Cap_Fade_2_40',
               'Intercept_Cap_Fade_2_40',
               'Diff_MaxCap_DC2',
               'Discharge_Cap_2',
               'Avg_charge_time',
               'Temp_Integ_2_40',
               'Min_IR',
               'Diff_IR_2_40']

In [27]:
#Use only the discharge model Tabular features
features = dismod_features #full_features
X_train=Training_Data[features].to_numpy()
X_test=Testing_Data[features].to_numpy()
X_sec=Secondary_Data[features].to_numpy()

#Scale
scaler = preprocessing.StandardScaler().fit(X_train)
Tabular_Train = scaler.transform(X_train)
Tabular_test = scaler.transform(X_test)
Tabular_sec = scaler.transform(X_sec)

In [28]:
#Load discharge capacity data
train_Qdlin=torch.load(r'data/DischargeCapacity-InterpolatedData_Train.pt')
test_Qdlin=torch.load(r'data/DischargeCapacity-InterpolatedData_Test.pt')
secondary_test_Qdlin=torch.load(r'data/DischargeCapacity-InterpolatedData_Secondary_Test.pt')

train_Qdlin = train_Qdlin[:,:40,:]
test_Qdlin = test_Qdlin[:,:40,:]
secondary_test_Qdlin = secondary_test_Qdlin[:,:40,:]

train_Qdlin = torch.permute(train_Qdlin, (0, 2, 1))
test_Qdlin = torch.permute(test_Qdlin, (0, 2, 1))
secondary_test_Qdlin = torch.permute(secondary_test_Qdlin, (0, 2, 1))

print(train_Qdlin.std(),train_Qdlin.mean())
print(test_Qdlin.std(),test_Qdlin.mean())
print(secondary_test_Qdlin.std(),secondary_test_Qdlin.mean())

print(train_Qdlin.shape)
print(test_Qdlin.shape)
print(secondary_test_Qdlin.shape)

tensor(1.1242) tensor(-0.6322)
tensor(0.9673) tensor(-0.5410)
tensor(0.5148) tensor(-0.2270)
torch.Size([41, 99, 40])
torch.Size([42, 99, 40])
torch.Size([40, 99, 40])


In [29]:
y_train=Training_Data[target_features].to_numpy()
y_test=Testing_Data[target_features].to_numpy()
y_sec=Secondary_Data[target_features].to_numpy()

y_train = torch.tensor(y_train)
y_test = torch.tensor(y_test)
y_sec = torch.tensor(y_sec)

y_train_mean = torch.mean(y_train)
y_train = y_train - y_train_mean
y_test = y_test - y_train_mean
y_sec = y_sec - y_train_mean
#Be sure to add y_train_mean and 10** All results

In [30]:
torch.set_default_dtype(torch.float64)

In [31]:
train_Dataset = CustomDataset(train_Qdlin,y_train)
test_Dataset = CustomDataset(test_Qdlin,y_test)
secondary_test_Dataset = CustomDataset(secondary_test_Qdlin,y_sec)

train_Qdlin_loader = DataLoader(train_Dataset, batch_size=41, shuffle=True)
test_Qdlin_loader = DataLoader(test_Dataset, batch_size=42, shuffle=True)
secondary_test_Qdlin_loader = DataLoader(secondary_test_Dataset, batch_size=40, shuffle=True)

# Hyper-parameters

In [32]:
hidden_size = 128 
hidden_size2 = 256
num_layers = 1
dropout = .2
num_dense_neurons= 32

epochs = 1500
lr = 0.0001
weight_decay = 0.001

In [33]:
input_size=train_Qdlin.shape[2] #FEATURES
num_cycles=train_Qdlin.shape[1] #TIMESTEPS(Cycles)

#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")
print(device)

cuda:0


# Train & Test

In [34]:
criterion=nn.MSELoss()
model=LSTMRNN(input_size, hidden_size, hidden_size2, num_layers,dropout, num_dense_neurons)
optimizer=optim.Adam(model.parameters(),weight_decay=weight_decay, lr=lr)
model = model.to(device)
model = train(model,train_Qdlin_loader,criterion,optimizer,epochs,device)

torch.Size([41, 99, 40])
torch.Size([41, 99, 128])
torch.Size([41, 99, 256])
torch.Size([41, 256])
-- 0 -epoch--RMSE--  387.82345400475833
-- 100 -epoch--RMSE--  231.02371741920018
-- 200 -epoch--RMSE--  161.34816570365788
-- 300 -epoch--RMSE--  117.59941719377098
-- 400 -epoch--RMSE--  97.1692954897887
-- 500 -epoch--RMSE--  83.92618162525962
-- 600 -epoch--RMSE--  75.16752533875761
-- 700 -epoch--RMSE--  70.29143260286499
-- 800 -epoch--RMSE--  65.6664380335742
-- 900 -epoch--RMSE--  60.81222405996365
-- 1000 -epoch--RMSE--  57.321212439421764
-- 1100 -epoch--RMSE--  55.91336577830687
-- 1200 -epoch--RMSE--  52.41775168707287
-- 1300 -epoch--RMSE--  50.814699551522054
-- 1400 -epoch--RMSE--  50.270284811912354


In [35]:
testALL(model)

49.102041404710235 ---RMSE---Train
127.33127071883881 ---RMSE---Test
218.38383932885077 ---RMSE---Test Secondary


################################################################################

# Repeat using only the first 20 cycles

################################################################################

In [36]:
#Load tabular data
Dataset=pd.read_csv(r'data/TabularSummaryBatteryData-20Cycles.csv')

# Scale target Battery life By log 10
Dataset['cell_life']=np.log10(Dataset['cell_life'])

Training_Data=Dataset.iloc[train_ind]
Testing_Data=Dataset.iloc[test_edited_ind]
Secondary_Data=Dataset.iloc[secondary_test_ind]

#choosing model features According to table 1
target_features=['cell_life']

varmod_features=['variance_dQ_20_10']

dismod_features=['minimum_dQ_20_10',
                 'variance_dQ_20_10',
                 'skewness_dQ_20_10',
                 'kurtosis_dQ_20_10',
                 'Discharge_Cap_2',
                 'Diff_MaxCap_DC2']
    
full_features=['minimum_dQ_20_10',
               'variance_dQ_20_10',
               'Slope_Cap_Fade_2_20',
               'Intercept_Cap_Fade_2_20',
               'Diff_MaxCap_DC2',
               'Discharge_Cap_2',
               'Avg_charge_time',
               'Temp_Integ_2_20',
               'Min_IR',
               'Diff_IR_2_20']

In [37]:
#Use only the discharge model Tabular features
features = dismod_features #full_features
X_train=Training_Data[features].to_numpy()
X_test=Testing_Data[features].to_numpy()
X_sec=Secondary_Data[features].to_numpy()

#Scale
scaler = preprocessing.StandardScaler().fit(X_train)
Tabular_Train = scaler.transform(X_train)
Tabular_test = scaler.transform(X_test)
Tabular_sec = scaler.transform(X_sec)

In [38]:
#Load discharge capacity data
train_Qdlin=torch.load(r'data/DischargeCapacity-InterpolatedData_Train.pt')
test_Qdlin=torch.load(r'data/DischargeCapacity-InterpolatedData_Test.pt')
secondary_test_Qdlin=torch.load(r'data/DischargeCapacity-InterpolatedData_Secondary_Test.pt')

train_Qdlin = train_Qdlin[:,:20,:]
test_Qdlin = test_Qdlin[:,:20,:]
secondary_test_Qdlin = secondary_test_Qdlin[:,:20,:]

train_Qdlin = torch.permute(train_Qdlin, (0, 2, 1))
test_Qdlin = torch.permute(test_Qdlin, (0, 2, 1))
secondary_test_Qdlin = torch.permute(secondary_test_Qdlin, (0, 2, 1))

print(train_Qdlin.std(),train_Qdlin.mean())
print(test_Qdlin.std(),test_Qdlin.mean())
print(secondary_test_Qdlin.std(),secondary_test_Qdlin.mean())

print(train_Qdlin.shape)
print(test_Qdlin.shape)
print(secondary_test_Qdlin.shape)

tensor(0.0264) tensor(-0.0102)
tensor(0.0276) tensor(-0.0079)
tensor(0.0151) tensor(0.0023)
torch.Size([41, 99, 20])
torch.Size([42, 99, 20])
torch.Size([40, 99, 20])


In [39]:
y_train=Training_Data[target_features].to_numpy()
y_test=Testing_Data[target_features].to_numpy()
y_sec=Secondary_Data[target_features].to_numpy()

y_train = torch.tensor(y_train)
y_test = torch.tensor(y_test)
y_sec = torch.tensor(y_sec)

y_train_mean = torch.mean(y_train)
y_train = y_train - y_train_mean
y_test = y_test - y_train_mean
y_sec = y_sec - y_train_mean
#Be sure to add y_train_mean and 10** All results

In [40]:
torch.set_default_dtype(torch.float64)

In [41]:
train_Dataset = CustomDataset(train_Qdlin,y_train)
test_Dataset = CustomDataset(test_Qdlin,y_test)
secondary_test_Dataset = CustomDataset(secondary_test_Qdlin,y_sec)

train_Qdlin_loader = DataLoader(train_Dataset, batch_size=41, shuffle=True)
test_Qdlin_loader = DataLoader(test_Dataset, batch_size=42, shuffle=True)
secondary_test_Qdlin_loader = DataLoader(secondary_test_Dataset, batch_size=40, shuffle=True)

# Hyper-parameters

In [42]:
hidden_size = 128 
hidden_size2 = 256
num_layers = 1
dropout = .2
num_dense_neurons= 32

epochs = 1500
lr = 0.0001
weight_decay = 0.001

In [43]:
input_size=train_Qdlin.shape[2] #FEATURES
num_cycles=train_Qdlin.shape[1] #TIMESTEPS(Cycles)

#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")
print(device)

cuda:0


# Train & Test

In [44]:
criterion=nn.MSELoss()
model=LSTMRNN(input_size, hidden_size, hidden_size2, num_layers,dropout, num_dense_neurons)
optimizer=optim.Adam(model.parameters(),weight_decay=weight_decay, lr=lr)
model = model.to(device)
model = train(model,train_Qdlin_loader,criterion,optimizer,epochs,device)

torch.Size([41, 99, 20])
torch.Size([41, 99, 128])
torch.Size([41, 99, 256])
torch.Size([41, 256])
-- 0 -epoch--RMSE--  406.70545410406277
-- 100 -epoch--RMSE--  336.972133521836
-- 200 -epoch--RMSE--  328.768479356721
-- 300 -epoch--RMSE--  327.93505631874046
-- 400 -epoch--RMSE--  327.40083276683
-- 500 -epoch--RMSE--  327.05558389324045
-- 600 -epoch--RMSE--  326.74161711126993
-- 700 -epoch--RMSE--  326.34575942098985
-- 800 -epoch--RMSE--  325.98953696199857
-- 900 -epoch--RMSE--  325.54888849489254
-- 1000 -epoch--RMSE--  324.843735394601
-- 1100 -epoch--RMSE--  322.77074457560974
-- 1200 -epoch--RMSE--  303.35357576383547
-- 1300 -epoch--RMSE--  292.0158526953501
-- 1400 -epoch--RMSE--  282.4533127902077


In [45]:
testALL(model)

271.0041192484168 ---RMSE---Train
352.45137838225827 ---RMSE---Test
281.1825573607863 ---RMSE---Test Secondary
